In [ ]:
#!/usr/bin/env python
# coding: utf-8
import sys
sys.path.append('..')
import umbrella_mesh
import elastic_rods
import linkage_vis
from umbrella_mesh import UmbrellaEnergyType
from bending_validation import suppress_stdout as so
from visualization_helper import *

import pipeline_helper, importlib, design_optimization_analysis
with so(): importlib.reload(pipeline_helper)
with so(): importlib.reload(design_optimization_analysis)

import pickle, gzip

from configuration import *

import py_newton_optimizer
OPTS = py_newton_optimizer.NewtonOptimizerOptions()
OPTS.gradTol = 1e-8
OPTS.verbose = 1
OPTS.beta = 1e-6
OPTS.niter = 300
OPTS.verboseNonPosDef = False

name = 'stadium_teaser'
input_path = '../../data/{}.json.gz'.format(name)

io, input_data, target_mesh, curr_um, thickness, target_height_multiplier = parse_input(input_path, handleBoundary=False, handlePivots = True)
rod_colors = get_color_field(curr_um, input_data)

def get_saddle_view(umbrella):
    import mesh
    view = linkage_vis.LinkageViewerWithSurface(umbrella, target_mesh, width=1024, height=600)
    set_surface_view_options(view, color = 'green', surface_color = 'gray', umbrella_transparent = False, surface_transparent = True)
    view.averagedMaterialFrames = True
    view.setCameraParams(((-9.084548137374812, -16.50668635107724, 10.928721819207178),
                         (0.2122869979916505, 0.45555144229916267, 0.8645271042037045),
                         (0.0, 0.0, 0.0)))
    view.showScalarField(rod_colors)
    return view

In [ ]:
curr_um = pickle.load(gzip.open("../../output/stadium_teaser_optimized_rest_state_2022_01_25_14_52_target_height_factor_5.0.pkl.gz", 'r'))

In [ ]:
use_pin = False

driver = curr_um.centralJoint()
jdo = curr_um.dofOffsetForJoint(driver)
fixedVars = (list(range(jdo, jdo + 6)) if use_pin else []) + curr_um.rigidJointAngleDoFIndices()

In [ ]:
view = get_saddle_view(curr_um)
view.show()

In [ ]:
break_input_angle_symmetry(curr_um)

In [ ]:
def eqm_callback(prob, i):
    if (i % 1 == 0):
        view.showScalarField(rod_colors)

In [ ]:
configure_umbrella_pre_deployment(curr_um, thickness, target_height_multiplier)

In [ ]:
results = staged_deployment(curr_um, np.logspace(-5, 0, 6), eqm_callback, OPTS, fixedVars, elasticEnergyIncreaseFactorLimit = 1.5)